In [ ]:
from mat3ra.standata.materials import Materials
from mat3ra.made.material import Material

silicon = Material(Materials.get_by_name_first_match('Silicon'))



In [ ]:
from utils.visualize import visualize_materials
visualize_materials(silicon)

## Create a silicon nanowire

In [ ]:
from mat3ra.made.tools.modify import filter_by_cylinder, rotate, filter_by_box, add_vacuum_sides
from mat3ra.made.tools.build.slab import create_slab, SlabConfiguration


slab_config = SlabConfiguration(bulk=silicon, miller_indices=(1,1,0), thickness=2, vacuum=0, use_orthogonal_z=True, xy_supercell_matrix=[[3,0],[0,2]]) 
silicon_slab = create_slab(slab_config)

visualize_materials(silicon_slab)
visualize_materials(silicon_slab, rotation="-90x")

rotated_slab = rotate(silicon_slab, "z", -45)
silicon_nanowire = filter_by_box(rotated_slab, [0.2,0.28,0], [0.78,0.87,1])
visualize_materials(silicon_nanowire)
visualize_materials(silicon_nanowire, rotation="-90x")
nanowire_xy = rotate(silicon_nanowire, "z", -45)
# nanowire_xy = rotate(nanowire_xy, "x", 90, rotate_cell=True)
nanowire_xy = add_vacuum_sides(nanowire_xy, 10.0, on_x=True, on_y=True)
visualize_materials(nanowire_xy)
visualize_materials(nanowire_xy, rotation="-90x")

## Add passivation

In [ ]:
from mat3ra.made.tools.modify import add_vacuum
from mat3ra.made.tools.build.utils import merge_materials
from mat3ra.made.tools.build.passivation import PassivationConfiguration, create_passivation, SurfacePassivationBuilder, SurfacePassivationBuilderParameters
nanowire_xy = add_vacuum(nanowire_xy, 10.0, on_top=True, to_bottom=True)
params = SurfacePassivationBuilderParameters(
    shadowing_radius=3.9,
    bond_length=1.1
)
passivation_builder = SurfacePassivationBuilder(params)

nanowire_yz= rotate(nanowire_xy, "y", 90)
nanowire_xz =   rotate(nanowire_xy, "x", 90)

# visualize_materials([nanowire_yz, nanowire_xz])
# visualize_materials([nanowire_yz, nanowire_xz], rotation="-90x")

passivation_config_yz = PassivationConfiguration(
    slab=nanowire_yz,
    passivant="H",
    bond_length=1.1,
)

passivation_config_xz = PassivationConfiguration(
    slab=nanowire_xz,
    passivant="H",
    bond_length=1.1,
)

passivated_nanowire_yz = create_passivation(passivation_config_yz, passivation_builder)
visualize_materials(passivated_nanowire_yz)
visualize_materials(passivated_nanowire_yz, rotation="-90x")

passivated_nanowire_xz = create_passivation(passivation_config_xz, passivation_builder)
visualize_materials(passivated_nanowire_xz)
visualize_materials(passivated_nanowire_xz, rotation="-90x")

reversed_passivated_nanowire_xz = rotate(passivated_nanowire_xz, "x", -90)
reversed_passivated_nanowire_yz = rotate(passivated_nanowire_yz, "y", -90)

passivated_nanowire = merge_materials([reversed_passivated_nanowire_xz, reversed_passivated_nanowire_yz])
visualize_materials(passivated_nanowire)
visualize_materials(passivated_nanowire, rotation="-90x")


In [ ]:
from mat3ra.made.tools.build.passivation import PassivationConfiguration, CoordinationBasedPassivationBuilder, create_passivation, get_unique_coordination_numbers
from mat3ra.made.tools.build.passivation.builders import CoordinationBasedPassivationBuilderParameters
nanowire = nanowire_xy
# nanowire_xy = add_vacuum(nanowire_xy, -10.0, on_top=True, to_bottom=True)
passivation_config = PassivationConfiguration(
    slab=nanowire,
    passivant="H",
    bond_length=1.1,
)

params = CoordinationBasedPassivationBuilderParameters(shadowing_radius=2.4, coordination_threshold=4)
passivation_builder = CoordinationBasedPassivationBuilder(params)
print(get_unique_coordination_numbers(passivation_config, params))

passivated_nanowire = create_passivation(passivation_config, passivation_builder)

visualize_materials(passivated_nanowire)
visualize_materials(passivated_nanowire, rotation="-90x")
visualize_materials(passivated_nanowire, rotation="-90x,90y")

In [ ]:
from utils.jupyterlite import set_materials
passivated_nanowire.name = "H-Passivated Silicon Nanowire"
set_materials([passivated_nanowire])